# Patient Management System

In [1]:
from pymongo import MongoClient
from datetime import datetime

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client["patient_db"]
patients_collection = db["patients"] 

In [3]:
# function to check if the id input is an integer and return the valid patient_id.

def get_valid_patient_id():
    while True:
        patient_id = input("Enter patient ID: ")
        if patient_id.isdigit():
            return patient_id
        print("Invalid ID format. Please enter a valid integer ID.")

In [4]:
# function to check if the integer id input is unique and return the unique patient_id. 

def get_unique_id():
    while True:
        patient_id = get_valid_patient_id()
        if patients_collection.find_one({"id": patient_id}):
            print(f"Error: Patient ID {patient_id} already exists. Please enter a unique ID.")
            continue
        return patient_id

In [5]:
# function to check if the year input is integer and return the valid year input. 

def get_valid_year():
    while True:
        year = input("Enter year: ").strip()
        if year.isdigit():
            return int(year) 
        print("Invalid year format. Please enter a valid integer year.")

In [6]:
# function to add the information of a patient 
# Aruguments : patient_id, first_name, last_name, date_of_birth ( 4 entries of information of the patient)
# It checks the validity of the data input, return success message. 
# Return error message if it fails.

def add_patient(patient_id, first_name, last_name, date_of_birth):
    while True: # checking validity of the birth date 
        try:
            birth_date = datetime.strptime(date_of_birth, "%Y-%m-%d")
            birth_date_str = birth_date.strftime("%Y-%m-%d") 
            break 
        except ValueError:
            print("Invalid date format. Please enter in YYYY-MM-DD format.")
            date_of_birth = input("Re-enter birth date: ")  
            
    patient = {
        "id": patient_id,
        "first_name": first_name,
        "last_name": last_name,
        "birth_date": birth_date_str
    }
    
    try:
        result = patients_collection.insert_one(patient)
        print(f"Patient added successfully with ID: {patient_id}")
    except Exception as e:
        print(f"Database error: {e}")
        return None

In [7]:
# function to list all the information of patients in the collection.
# If there is no data found, print not found message.

def list_patients(query={}):

    for patient in patients_collection.find(query):
        print(f"ID: {patient['id']}")
        print(f"Fist Name: {patient['first_name']}")
        print(f"Last Name: {patient['last_name']}")
        print(f"Date of Birth: {patient['birth_date']}")
        print("-" * 20)
    if patients_collection.count_documents(query) == 0:
        print("No patients found")

In [8]:
# function to add the information of a patient  
# Aruguments : patient_id of the patient
# If the id is found, return the information of the patient. If not, return not found message. 

def find_patient_by_id(patient_id):     
    result = patients_collection.find_one({"id": patient_id})
    if result is None:
        print("No patients found")
    else:
        print(f"ID: {result['id']}")
        print(f"Fist Name: {result['first_name']}")
        print(f"Last Name: {result['last_name']}")
        print(f"Date of Birth: {result['birth_date']}")
        print("-" * 20)

In [9]:
# function to update the last name of a patient 
# Aruguments : patient ID and new last name of the patient
# If the id is found, update the last name of the patient. If not, return not found message.
# Return message to inform success or failure of the action at the end.

def update_patient_last_name(patient_id, new_last_name):
    result = patients_collection.find_one({"id": patient_id})
    if result is None:
        print("No patients found")
    else:
        update_result = patients_collection.update_one(
            {"id": patient_id}, 
            {"$set": {"last_name": new_last_name}} 
        )
    if update_result.modified_count > 0:
            print(f"Patient ID {patient_id}'s last name updated to {new_last_name}.")
    else:
            print(f"Update failed or last name was already '{new_last_name}'.") 

In [10]:
# function to update find patients born after a certain year.
# Aruguments : year 
# If there are patients born after the given year, return the list of patients. 
# If not, return not found message.

def find_patients_born_after_year(year):
    patients = patients_collection.find({"birth_date": {"$gt": f"{year}-01-01"}})
    found = False  
    for patient in patients:
        found = True
        print(f"ID: {patient['id']}")
        print(f"First Name: {patient['first_name']}")
        print(f"Last Name: {patient['last_name']}")
        print(f"Date of Birth: {patient['birth_date']}")
        print("-" * 20)
    if not found:
        print("No patients found born after", year)

In [11]:
# function to add the contact information of a patient.
# Aruguments : patient_id, email, phone 
# If the id is found, update the information of the patient, return success message.
# If not, return not found message. 

def add_patient_contact(patient_id, email, phone):
    result = patients_collection.find_one({"id": patient_id})
    if result is None:
        print("No patients found")
    else:
        update_result = patients_collection.update_one(
            {"id": patient_id}, 
            {"$set": {"email": email, "phone": phone}} 
        )
    if update_result.modified_count > 0:
        print(f"Patient ID {patient_id} contact information updated.")
    else:
        print(f"No changes made. Email and phone may already be the same.")

In [14]:
# function to delete information of a patient.
# Aruguments : patient_id
# If the id is found, delete the information of the patient, return success message.
# If not, return not found message. 

def delete_patient_by_id(patient_id):
    delete_result = patients_collection.delete_one({"id": patient_id})
    if delete_result.deleted_count > 0:
        print(f"Patient ID {patient_id} deleted successfully.")
    else:
        print(f"No patient found with ID: {patient_id}")

In [ ]:
def main():
    while True: #print out the action menu until exit action is taken.
        print("\nOptions:")
        print("1. Add a patient")
        print("2. List all patients")
        print("3. Find a patient by ID")
        print("4. Update patient's last name")
        print("5. Find patients born after a given year")
        print("6. Add patient contact info")
        print("7. Delete a patient")
        print("8. Exit")

        choice = input("Enter your choice: ")
        
        if choice == "1": # prompt the information input of the patient and add a patient
            patient_id = get_unique_id()
            first_name = input("Enter first name of the patient: ")
            last_name = input("Enter last name of the patient: ")
            date_of_birth = input("Enter Birth date of the patient: ")
            add_patient(patient_id, first_name, last_name, date_of_birth)
        elif choice == "2": # list all patients 
            list_patients()
        elif choice == "3": # find a patient by ID 
            patient_id = get_valid_patient_id()
            find_patient_by_id(patient_id)
        elif choice == "4": # update patient's last name
            patient_id = get_valid_patient_id()
            new_last_name = input("Enter patient's new last name to update : ")
            update_patient_last_name(patient_id, new_last_name)
        elif choice == "5": # find patients born after a given year 
            year = get_valid_year()
            find_patients_born_after_year(year)
        elif choice == "6": # add patient's contact info
            patient_id = get_valid_patient_id()
            email = input("Enter patient's email: ")
            phone = input("Enter phone number: ")
            add_patient_contact(patient_id, email, phone)
        elif choice == "7": # delete a patient 
            patient_id = get_valid_patient_id()
            delete_patient_by_id(patient_id)
        elif choice == "8": # exit 
            break
        else:
            print("Invalid choice.")



if __name__ == "__main__":
    main()


Options:
1. Add a patient
2. List all patients
3. Find a patient by ID
4. Update patient's last name
5. Find patients born after a given year
6. Add patient contact info
7. Delete a patient
8. Exit
